# AMP-SOA portfolio optimization using adaptive meta-policy based on second-order agents with deep reinforcment learning
---

## 5.0 Feature Reduction
---

Autoencoders are used to reduce the number of technical indicators to be represented by only two features

### 5.1 Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from numpy import array
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from sklearn import preprocessing

### 5.2 Load the Data

In [2]:
df = pd.read_csv('./df.csv')
%store -r df

In [3]:
df.head()

,date,tic,close,high,low,open,volume,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi,cov_list
0,2009-03-20,AAPL,3.067483,3.6825,3.591786,3.646071,695587200.0,21.624245,240.771026,-2.878423e+11,-15.604885,-1.092894,4.784446,17.715581,18.197433,-112.331634,45.730861,"[[0.0013155151011021062, 0.0011575384750591776..."
1,2009-03-20,AXP,9.629786,13.1900,12.120000,13.190000,31088200.0,20.474072,241.683988,-2.878112e+11,-15.658606,-1.695398,4.652402,16.456213,16.879334,-74.319369,47.538704,"[[0.0013155151011021062, 0.0011575384750591776..."
2,2009-03-20,CAT,17.987209,28.9000,26.730000,28.629999,16531300.0,20.353686,242.338441,-2.877947e+11,-15.545789,-1.481436,4.441961,16.727739,17.049776,-15.127649,49.830865,"[[0.0013155151011021062, 0.0011575384750591776..."
3,2009-03-20,CSCO,10.697708,16.5700,15.750000,16.370001,66078200.0,18.542039,242.632129,-2.878608e+11,-15.336297,-1.878417,4.241299,16.557303,16.072535,-61.675433,47.866388,"[[0.0013155151011021062, 0.0011575384750591776..."
4,2009-03-20,HD,15.282681,22.7300,21.760000,22.590000,22361800.0,17.891064,242.822755,-2.878384e+11,-15.196946,-1.802284,3.962916,17.226666,15.951019,-36.079148,49.222342,"[[0.0013155151011021062, 0.0011575384750591776..."


In [4]:
# Get the list of all the features
features_list = list(df.columns)[7:-1]

In [5]:
print(features_list)

['atr', 'bbw', 'obv', 'cmf', 'macd', 'adx', 'sma', 'ema', 'cci', 'rsi']


### 5.2 Construct a Data Frame of Features

In [6]:
# construct a data frame of features
features_df = df[features_list]
features_df.index = df['date']

In [7]:
features_df.head()

,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
date,,,,,,,,,,
2009-03-20,21.624245,240.771026,-2.878423e+11,-15.604885,-1.092894,4.784446,17.715581,18.197433,-112.331634,45.730861
2009-03-20,20.474072,241.683988,-2.878112e+11,-15.658606,-1.695398,4.652402,16.456213,16.879334,-74.319369,47.538704
2009-03-20,20.353686,242.338441,-2.877947e+11,-15.545789,-1.481436,4.441961,16.727739,17.049776,-15.127649,49.830865
2009-03-20,18.542039,242.632129,-2.878608e+11,-15.336297,-1.878417,4.241299,16.557303,16.072535,-61.675433,47.866388
2009-03-20,17.891064,242.822755,-2.878384e+11,-15.196946,-1.802284,3.962916,17.226666,15.951019,-36.079148,49.222342


In [8]:
features_df.shape

(75640, 10)

In [9]:
df.shape

(75640, 18)

### 5.3 Normalize the Features and Construct an Autoencoder

In [10]:
features_array = np.array(features_df)

features_scaler = preprocessing.MinMaxScaler()
features_normalised = features_scaler.fit_transform(features_array)

In [11]:
features_normalised.shape

(75640, 10)

In [12]:
# rescale the features array
features_normalised = features_normalised.reshape(-1,20,10)

In [13]:
features_normalised.shape

(3782, 20, 10)

In [14]:
# define model
model = Sequential()
model.add(LSTM(4, activation='relu', input_shape=(20,10)))
model.add(RepeatVector(20))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(10)))
model.compile(optimizer='adam', loss='mse')

In [15]:
# fit model
model.fit(features_normalised, features_normalised, epochs=200, verbose=1)
#plot_model(model, show_shapes=True, to_file='./results/reconstruct_lstm_autoencoder.png')

Epoch 1/200

119/119 [==============================] - 5s 16ms/step - loss: 0.0706
Epoch 2/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0137
Epoch 3/200
119/119 [==============================] - 2s 17ms/step - loss: 0.0087
Epoch 4/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0072
Epoch 5/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0052
Epoch 6/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0042
Epoch 7/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0037
Epoch 8/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0034
Epoch 9/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0030
Epoch 10/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0024
Epoch 11/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0020
Epoch 12/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0018


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 240       
                                                                 
 repeat_vector (RepeatVecto  (None, 20, 4)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 20, 50)            11000     
                                                                 
 lstm_2 (LSTM)               (None, 20, 100)           60400     
                                                                 
 time_distributed (TimeDist  (None, 20, 10)            1010      
 ributed)                                                        
                                                                 
Total params: 72650 (283.79 KB)
Trainable params: 72650 

In [17]:
# connect the encoder LSTM as the output layer
model_feature = Model(inputs=model.inputs, outputs=model.layers[1].output)
#plot_model(model_feature, show_shapes=True, show_layer_names=True, to_file='./results/lstm_encoder.png')

In [18]:
model_feature.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 20, 10)]          0         
                                                                 
 lstm (LSTM)                 (None, 4)                 240       
                                                                 
 repeat_vector (RepeatVecto  (None, 20, 4)             0         
 r)                                                              
                                                                 
Total params: 240 (960.00 Byte)
Trainable params: 240 (960.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 5.4 Get the Reconstructed Features

In [19]:
# get the feature vector for the input sequence
yhat = model_feature.predict(features_normalised)
print(yhat.shape)

119/119 [==============================] - 0s 1ms/step
(3782, 20, 4)


In [20]:
# reshape the vector
features_reduced = yhat.reshape(-1,4)

In [21]:
df.shape

(75640, 18)

In [22]:
features_reduced.shape

(75640, 4)

In [23]:
# Copy original data frame and drop the original features
df_reduced = df.copy()
df_reduced = df_reduced .drop(features_list, axis=1)

In [24]:
df_reduced.head()

,date,tic,close,high,low,open,volume,cov_list
0,2009-03-20,AAPL,3.067483,3.6825,3.591786,3.646071,695587200.0,"[[0.0013155151011021062, 0.0011575384750591776..."
1,2009-03-20,AXP,9.629786,13.1900,12.120000,13.190000,31088200.0,"[[0.0013155151011021062, 0.0011575384750591776..."
2,2009-03-20,CAT,17.987209,28.9000,26.730000,28.629999,16531300.0,"[[0.0013155151011021062, 0.0011575384750591776..."
3,2009-03-20,CSCO,10.697708,16.5700,15.750000,16.370001,66078200.0,"[[0.0013155151011021062, 0.0011575384750591776..."
4,2009-03-20,HD,15.282681,22.7300,21.760000,22.590000,22361800.0,"[[0.0013155151011021062, 0.0011575384750591776..."


In [25]:
# convert the reduced features to a data frame and merge with the original data frame
features_reduced_df = pd.DataFrame(features_reduced, columns=['f01','f02','f03','f04'])

In [26]:
features_reduced_df.head()

,f01,f02,f03,f04
0,1.766427,4.765831,0.804321,1.373377
1,1.766427,4.765831,0.804321,1.373377
2,1.766427,4.765831,0.804321,1.373377
3,1.766427,4.765831,0.804321,1.373377
4,1.766427,4.765831,0.804321,1.373377


In [27]:
df_reduced[['f01','f02','f03','f04']] = features_reduced_df[['f01','f02','f03','f04']]

In [28]:
df_reduced.head()

,date,tic,close,high,low,open,volume,cov_list,f01,f02,f03,f04
0,2009-03-20,AAPL,3.067483,3.6825,3.591786,3.646071,695587200.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
1,2009-03-20,AXP,9.629786,13.1900,12.120000,13.190000,31088200.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
2,2009-03-20,CAT,17.987209,28.9000,26.730000,28.629999,16531300.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
3,2009-03-20,CSCO,10.697708,16.5700,15.750000,16.370001,66078200.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
4,2009-03-20,HD,15.282681,22.7300,21.760000,22.590000,22361800.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377


In [29]:
data_df = df_reduced.copy()

In [30]:
%store data_df
data_df.to_csv('./data_df.csv')


Stored 'data_df' (DataFrame)
